# Imports and Setup

In [1]:

!pip install --upgrade google-cloud-bigquery google-cloud-aiplatform db-dtypes

In [2]:

from google.cloud import bigquery
from google.cloud import aiplatform
import pandas as pd
from IPython.display import display, Markdown

# Initialize clients
project_id = "qwiklabs-gcp-01-34739c20280a"
location = "us-central1"

# Initialize BigQuery client
bq_client = bigquery.Client(project=project_id, location=location)

# Initialize Vertex AI
aiplatform.init(project=project_id, location=location)

print(f" Project ID: {project_id}")
print(f" Location: {location}")
print(f" BigQuery client initialized")

/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


 Project ID: qwiklabs-gcp-01-34739c20280a
 Location: us-central1
 BigQuery client initialized


In [3]:

dataset_id = "aurora_bay_faqs"
table_name = "aurora_bay_faqs_raw"
embeddings_table = "aurora_bay_faqs_with_embeddings"
embedding_model_name = "embedding_model"
gemini_model_name = "gemini_model"

# Full resource names
dataset_full = f"{project_id}.{dataset_id}"
embedding_model_full = f"{dataset_full}.{embedding_model_name}"
gemini_model_full = f"{dataset_full}.{gemini_model_name}"

print(f" Dataset: {dataset_full}")
print(f" Raw table: {dataset_full}.{table_name}")
print(f" Embeddings table: {dataset_full}.{embeddings_table}")
print(f" Embedding model: {embedding_model_full}")
print(f" Gemini model: {gemini_model_full}")

 Dataset: qwiklabs-gcp-01-34739c20280a.aurora_bay_faqs
 Raw table: qwiklabs-gcp-01-34739c20280a.aurora_bay_faqs.aurora_bay_faqs_raw
 Embeddings table: qwiklabs-gcp-01-34739c20280a.aurora_bay_faqs.aurora_bay_faqs_with_embeddings
 Embedding model: qwiklabs-gcp-01-34739c20280a.aurora_bay_faqs.embedding_model
 Gemini model: qwiklabs-gcp-01-34739c20280a.aurora_bay_faqs.gemini_model


In [4]:

dataset = bigquery.Dataset(dataset_full)
dataset.location = location

try:
    dataset = bq_client.create_dataset(dataset, exists_ok=True)
    print(f" Dataset {dataset_id} created or already exists")
except Exception as e:
    print(f" Error creating dataset: {e}")

 Dataset aurora_bay_faqs created or already exists


## Verify Connection

In [5]:

# Update location variable
location = "us"  # Changed from us-central1 to multi-region US

print(f" Updated location to: {location}")
print(f" Connection should be: {location}.vertex-ai")

 Updated location to: us
 Connection should be: us.vertex-ai


In [6]:
# Verify connection exists
connection_id = f"{location}.vertex-ai"

print(f"Checking for connection: {connection_id}")

# Simple check - try to use it
print(" Assuming connection created successfully in BigQuery Console")
print(f" Connection: {connection_id}")

Checking for connection: us.vertex-ai
 Assuming connection created successfully in BigQuery Console
 Connection: us.vertex-ai


Manually added a connection in BQ to Vertex AI

In [7]:

dataset_id = "aurora_bay_faqs"
location = "us"

# Delete old dataset if it exists
try:
    bq_client.delete_dataset(f"{project_id}.{dataset_id}", delete_contents=True, not_found_ok=True)
    print(f"🗑️ Deleted old dataset (if it existed)")
except Exception as e:
    print(f"Note: {e}")

# Create new dataset in US region
dataset_full = f"{project_id}.{dataset_id}"
dataset = bigquery.Dataset(dataset_full)
dataset.location = "us"  # Important: must match connection location

dataset = bq_client.create_dataset(dataset, exists_ok=True)
print(f" Dataset created in location: {dataset.location}")
print(f" Dataset: {dataset_full}")

# Update all variables
embedding_model_full = f"{dataset_full}.embedding_model"
gemini_model_full = f"{dataset_full}.gemini_model"

print(f" Embedding model: {embedding_model_full}")
print(f" Gemini model: {gemini_model_full}")

🗑️ Deleted old dataset (if it existed)
 Dataset created in location: US
 Dataset: qwiklabs-gcp-01-34739c20280a.aurora_bay_faqs
 Embedding model: qwiklabs-gcp-01-34739c20280a.aurora_bay_faqs.embedding_model
 Gemini model: qwiklabs-gcp-01-34739c20280a.aurora_bay_faqs.gemini_model


In [8]:
# Update BigQuery client with correct location
location = "us"
project_id = "qwiklabs-gcp-01-34739c20280a"

# Reinitialize BigQuery client with US location
bq_client = bigquery.Client(project=project_id, location=location)

print(f" BigQuery client location set to: {location}")
print(f" Project: {project_id}")

 BigQuery client location set to: us
 Project: qwiklabs-gcp-01-34739c20280a


In [9]:
# Verify dataset exists in US region
dataset_ref = f"{project_id}.aurora_bay_faqs"

try:
    dataset = bq_client.get_dataset(dataset_ref)
    print(f" Dataset found: {dataset_ref}")
    print(f" Dataset location: {dataset.location}")
except Exception as e:
    print(f" Error: {e}")

 Dataset found: qwiklabs-gcp-01-34739c20280a.aurora_bay_faqs
 Dataset location: US


In [10]:
#  Retry creating embedding model after permissions granted
create_embedding_model_query = f"""
CREATE OR REPLACE MODEL `{project_id}.aurora_bay_faqs.embedding_model`
REMOTE WITH CONNECTION `us.vertex-ai`
OPTIONS (ENDPOINT = 'text-embedding-005');
"""

print("Creating embedding model (after granting permissions)...")
print(create_embedding_model_query)

try:
    job = bq_client.query(create_embedding_model_query)
    job.result()
    print(" Embedding model created successfully!")
except Exception as e:
    print(f" Error: {e}")

Creating embedding model (after granting permissions)...

CREATE OR REPLACE MODEL `qwiklabs-gcp-01-34739c20280a.aurora_bay_faqs.embedding_model`
REMOTE WITH CONNECTION `us.vertex-ai`
OPTIONS (ENDPOINT = 'text-embedding-005');

 Embedding model created successfully!


In [11]:
# Load CSV from Cloud Storage into BigQuery
from google.cloud.bigquery import LoadJobConfig, SourceFormat

# Define table reference
table_id = f"{project_id}.aurora_bay_faqs.aurora_bay_faqs_raw"
gcs_uri = "gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv"

# Configure load job
job_config = LoadJobConfig(
    source_format=SourceFormat.CSV,
    skip_leading_rows=1,  # Skip header row
    autodetect=True,  # Auto-detect schema
    write_disposition="WRITE_TRUNCATE",  # Overwrite if exists
)

print(f"Loading data from: {gcs_uri}")
print(f"Into table: {table_id}")

try:
    load_job = bq_client.load_table_from_uri(
        gcs_uri,
        table_id,
        job_config=job_config
    )
    load_job.result()  # Wait for completion

    # Get table info
    table = bq_client.get_table(table_id)
    print(f" Loaded {table.num_rows} rows into {table_id}")
    print(f" Schema: {[field.name for field in table.schema]}")

except Exception as e:
    print(f" Error loading data: {e}")

Loading data from: gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv
Into table: qwiklabs-gcp-01-34739c20280a.aurora_bay_faqs.aurora_bay_faqs_raw
 Loaded 50 rows into qwiklabs-gcp-01-34739c20280a.aurora_bay_faqs.aurora_bay_faqs_raw
 Schema: ['string_field_0', 'string_field_1']


In [12]:
# Preview the loaded data
preview_query = f"""
SELECT *
FROM `{project_id}.aurora_bay_faqs.aurora_bay_faqs_raw`
LIMIT 5
"""

print("Preview of loaded FAQ data:")
preview_df = bq_client.query(preview_query).to_dataframe()
display(preview_df)

Preview of loaded FAQ data:


,string_field_0,string_field_1
0,When was Aurora Bay founded?,Aurora Bay was founded in 1901 by a group of f...
1,What is the population of Aurora Bay?,Aurora Bay has a population of approximately 3...
2,Where is the Aurora Bay Town Hall located?,The Town Hall is located at 100 Harbor View Ro...
3,Who is the current mayor of Aurora Bay?,"The current mayor is Linda Greenwood, elected ..."
4,What are the primary industries in Aurora Bay?,The primary industries include commercial fish...


In [13]:
# Check what the actual headers should be and create a proper table
# First, let's see the first row which likely has headers
check_query = f"""
SELECT *
FROM `{project_id}.aurora_bay_faqs.aurora_bay_faqs_raw`
LIMIT 1
"""

first_row = bq_client.query(check_query).to_dataframe()
print("First row (likely contains actual headers):")
display(first_row)

First row (likely contains actual headers):


,string_field_0,string_field_1
0,When was Aurora Bay founded?,Aurora Bay was founded in 1901 by a group of f...


In [14]:
# Create properly named table assuming columns are Question and Answer
create_proper_table = f"""
CREATE OR REPLACE TABLE `{project_id}.aurora_bay_faqs.aurora_bay_faqs` AS
SELECT
  string_field_0 AS question,
  string_field_1 AS answer
FROM `{project_id}.aurora_bay_faqs.aurora_bay_faqs_raw`
WHERE string_field_0 != 'Question'  -- Skip header row if it was included
  AND string_field_0 IS NOT NULL
"""

print("Creating properly named FAQ table...")
try:
    job = bq_client.query(create_proper_table)
    job.result()
    print(" Table created with proper column names!")

    # Preview new table
    preview = bq_client.query(f"""
        SELECT * FROM `{project_id}.aurora_bay_faqs.aurora_bay_faqs` LIMIT 3
    """).to_dataframe()
    display(preview)

except Exception as e:
    print(f" Error: {e}")

Creating properly named FAQ table...
 Table created with proper column names!


,question,answer
0,What types of recreation are available in Auro...,"Popular activities include fishing, kayaking, ..."
1,Does Aurora Bay offer snow removal services fo...,Yes. The Public Works Department clears main r...
2,What local outdoor adventure companies operate...,North Star Excursions and Bay Explorers offer ...


# Generate Embeddings

In [15]:
# Create table with embeddings
create_embeddings_table = f"""
CREATE OR REPLACE TABLE `{project_id}.aurora_bay_faqs.aurora_bay_faqs_with_embeddings` AS
SELECT *
FROM ML.GENERATE_EMBEDDING(
  MODEL `{project_id}.aurora_bay_faqs.embedding_model`,
  (
    SELECT
      question,
      answer,
      CONCAT(question, ' ', answer) AS content
    FROM `{project_id}.aurora_bay_faqs.aurora_bay_faqs`
  )
);
"""

print("Generating embeddings for all FAQ entries...")
print("This may take a minute...")
print(create_embeddings_table)

try:
    job = bq_client.query(create_embeddings_table)
    job.result()  # Wait for completion
    print(" Embeddings table created successfully!")

    # Check the table
    table = bq_client.get_table(f"{project_id}.aurora_bay_faqs.aurora_bay_faqs_with_embeddings")
    print(f" Total rows: {table.num_rows}")
    print(f" Columns: {[field.name for field in table.schema]}")

except Exception as e:
    print(f" Error: {e}")

Generating embeddings for all FAQ entries...
This may take a minute...

CREATE OR REPLACE TABLE `qwiklabs-gcp-01-34739c20280a.aurora_bay_faqs.aurora_bay_faqs_with_embeddings` AS
SELECT *
FROM ML.GENERATE_EMBEDDING(
  MODEL `qwiklabs-gcp-01-34739c20280a.aurora_bay_faqs.embedding_model`,
  (
    SELECT
      question,
      answer,
      CONCAT(question, ' ', answer) AS content
    FROM `qwiklabs-gcp-01-34739c20280a.aurora_bay_faqs.aurora_bay_faqs`
  )
);

 Embeddings table created successfully!
 Total rows: 50
 Columns: ['ml_generate_embedding_result', 'ml_generate_embedding_statistics', 'ml_generate_embedding_status', 'question', 'answer', 'content']


In [16]:
# Preview embeddings table
preview_embeddings = f"""
SELECT
  question,
  answer,
  content,
  ARRAY_LENGTH(ml_generate_embedding_result) AS embedding_dimension
FROM `{project_id}.aurora_bay_faqs.aurora_bay_faqs_with_embeddings`
LIMIT 3
"""

print("Preview of embeddings table:")
preview_df = bq_client.query(preview_embeddings).to_dataframe()
display(preview_df)

Preview of embeddings table:


,question,answer,content,embedding_dimension
0,What are the primary industries in Aurora Bay?,The primary industries include commercial fish...,What are the primary industries in Aurora Bay?...,768
1,When was Aurora Bay founded?,Aurora Bay was founded in 1901 by a group of f...,When was Aurora Bay founded? Aurora Bay was fo...,768
2,Who is the current mayor of Aurora Bay?,"The current mayor is Linda Greenwood, elected ...",Who is the current mayor of Aurora Bay? The cu...,768


# Gemini Model

In [17]:
# Preview embeddings table
preview_embeddings = f"""
SELECT
  question,
  answer,
  content,
  ARRAY_LENGTH(ml_generate_embedding_result) AS embedding_dimension
FROM `{project_id}.aurora_bay_faqs.aurora_bay_faqs_with_embeddings`
LIMIT 3
"""

print("Preview of embeddings table:")
preview_df = bq_client.query(preview_embeddings).to_dataframe()
display(preview_df)

Preview of embeddings table:


,question,answer,content,embedding_dimension
0,What are the primary industries in Aurora Bay?,The primary industries include commercial fish...,What are the primary industries in Aurora Bay?...,768
1,When was Aurora Bay founded?,Aurora Bay was founded in 1901 by a group of f...,When was Aurora Bay founded? Aurora Bay was fo...,768
2,Who is the current mayor of Aurora Bay?,"The current mayor is Linda Greenwood, elected ...",Who is the current mayor of Aurora Bay? The cu...,768


In [18]:
# Verify and create Gemini model
create_gemini_model = f"""
CREATE OR REPLACE MODEL `{project_id}.aurora_bay_faqs.gemini_model`
REMOTE WITH CONNECTION `us.vertex-ai`
OPTIONS (ENDPOINT = 'gemini-2.0-flash');
"""

print("Creating Gemini model...")
print(create_gemini_model)

try:
    job = bq_client.query(create_gemini_model)
    job.result()
    print(" Gemini model created successfully!")

    # Verify it exists
    model = bq_client.get_model(f"{project_id}.aurora_bay_faqs.gemini_model")
    print(f" Model verified: {model.model_reference}")

except Exception as e:
    print(f" Error: {e}")

Creating Gemini model...

CREATE OR REPLACE MODEL `qwiklabs-gcp-01-34739c20280a.aurora_bay_faqs.gemini_model`
REMOTE WITH CONNECTION `us.vertex-ai`
OPTIONS (ENDPOINT = 'gemini-2.0-flash');

 Gemini model created successfully!
 Error: 'Model' object has no attribute 'model_reference'


# Vector Search

In [19]:
# Function to search FAQs using vector similarity
def search_faqs(user_question, top_k=5):
    """
    Search for relevant FAQs using vector similarity

    Args:
        user_question: The user's question
        top_k: Number of results to return

    Returns:
        DataFrame with matching FAQs
    """

    search_query = f"""
    SELECT
      query.query AS user_question,
      base.question AS faq_question,
      base.answer AS faq_answer
    FROM
      VECTOR_SEARCH(
        TABLE `{project_id}.aurora_bay_faqs.aurora_bay_faqs_with_embeddings`,
        'ml_generate_embedding_result',
        (
          SELECT
            text_embedding,
            content AS query
          FROM
            ML.GENERATE_TEXT_EMBEDDING(
              MODEL `{project_id}.aurora_bay_faqs.embedding_model`,
              (
                SELECT '{user_question}' AS content
              )
            )
        ),
        top_k => {top_k},
        OPTIONS => '{{"fraction_lists_to_search": 0.01}}'
      )
    """

    try:
        results = bq_client.query(search_query).to_dataframe()
        return results
    except Exception as e:
        print(f" Search error: {e}")
        return None

print("Vector search function created!")

Vector search function created!


In [20]:
# Test the vector search
test_question = "Where is the town hall?"

print(f"Testing search with question: '{test_question}'")
print("-" * 80)

search_results = search_faqs(test_question, top_k=3)

if search_results is not None:
    print(f"Found {len(search_results)} relevant FAQs:")
    display(search_results)

Testing search with question: 'Where is the town hall?'
--------------------------------------------------------------------------------
Found 3 relevant FAQs:


,user_question,faq_question,faq_answer
0,Where is the town hall?,Where is the Aurora Bay Town Hall located?,The Town Hall is located at 100 Harbor View Ro...
1,Where is the town hall?,When are the town council meetings held?,Town council meetings are held every second Tu...
2,Where is the town hall?,How do I request a building permit?,Building permit applications can be obtained a...


# RAG Chatbot

In [21]:

import time

def ask_aurora_bay(user_question, top_k=5):
    """
    RAG chatbot for Aurora Bay FAQs

    Args:
        user_question: The user's question
        top_k: Number of FAQs to retrieve for context

    Returns:
        Generated answer from Gemini
    """

    # Step 1: Search for relevant FAQs
    print(f" Searching for relevant information...")
    search_results = search_faqs(user_question, top_k=top_k)

    if search_results is None or len(search_results) == 0:
        return "I couldn't find relevant information to answer your question."

    # Step 2: Build context from search results
    context = "\n\n".join([
        f"Q: {row['faq_question']}\nA: {row['faq_answer']}"
        for _, row in search_results.iterrows()
    ])

    print(f" Found {len(search_results)} relevant FAQs")
    print(f"\n Building prompt with context...")

    # Step 3: Create prompt for Gemini
    prompt = f"""You are a helpful assistant for Aurora Bay, Alaska. Answer the user's question based ONLY on the following context from Aurora Bay FAQs. If the answer is not in the context, say you don't have that information.

Context:
{context}

User Question: {user_question}

Answer:"""

    # Step 4: Create a temporary table with the prompt
    print(f" Generating answer with Gemini...")

    # Create temp table with prompt
    temp_table_id = f"{project_id}.aurora_bay_faqs.temp_prompt_{int(time.time())}"

    prompt_df = pd.DataFrame({'prompt': [prompt]})

    try:
        # Load prompt to temp table
        job = bq_client.load_table_from_dataframe(prompt_df, temp_table_id)
        job.result()

        # Query Gemini using the temp table
        gemini_query = f"""
        SELECT
          ml_generate_text_result['candidates'][0]['content']['parts'][0]['text'] AS generated_answer
        FROM
          ML.GENERATE_TEXT(
            MODEL `{project_id}.aurora_bay_faqs.gemini_model`,
            (SELECT prompt FROM `{temp_table_id}`),
            STRUCT(
              0.2 AS temperature,
              1024 AS max_output_tokens
            )
          )
        """

        result = bq_client.query(gemini_query).to_dataframe()
        answer = result['generated_answer'].iloc[0]

        # Clean up temp table
        bq_client.delete_table(temp_table_id, not_found_ok=True)

        return answer

    except Exception as e:
        print(f" Error generating answer: {e}")
        # Clean up temp table
        try:
            bq_client.delete_table(temp_table_id, not_found_ok=True)
        except:
            pass
        return f"Error: {e}"

print(" RAG Chatbot function created!")

 RAG Chatbot function created!


In [22]:
# Test the complete RAG chatbot
test_questions = [
    "Where is the Aurora Bay Town Hall located?",
    "Who is the mayor?",
    "What are the main industries?"
]

for question in test_questions:
    print("=" * 80)
    print(f" Question: {question}")
    print("-" * 80)

    answer = ask_aurora_bay(question, top_k=3)

    print(f"\n Answer:\n{answer}")
    print()

 Question: Where is the Aurora Bay Town Hall located?
--------------------------------------------------------------------------------
 Searching for relevant information...
 Found 3 relevant FAQs

 Building prompt with context...
 Generating answer with Gemini...

 Answer:
"The Town Hall is located at 100 Harbor View Road, in the center of Aurora Bay, close to the main harbor.\n"

 Question: Who is the mayor?
--------------------------------------------------------------------------------
 Searching for relevant information...
 Found 3 relevant FAQs

 Building prompt with context...
 Generating answer with Gemini...

 Answer:
"The current mayor is Linda Greenwood, elected in 2021 for a four-year term.\n"

 Question: What are the main industries?
--------------------------------------------------------------------------------
 Searching for relevant information...
 Found 3 relevant FAQs

 Building prompt with context...
 Generating answer with Gemini...

 Answer:
"The primary industrie

# Interactive Chatbot

In [23]:
# Interactive chatbot function
def chat_with_aurora_bay():
    """
    Interactive chatbot session
    """
    print("=" * 80)
    print(" Welcome to Aurora Bay FAQ Chatbot!")
    print("=" * 80)
    print("Ask me anything about Aurora Bay, Alaska.")
    print("Type 'quit' or 'exit' to end the conversation.")
    print("=" * 80)
    print()

    while True:
        user_input = input("You: ").strip()

        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("\n Thank you for using Aurora Bay FAQ Chatbot!")
            break

        if not user_input:
            continue

        print()
        answer = ask_aurora_bay(user_input, top_k=5)
        print(f"\n Aurora Bay Bot:\n{answer}\n")
        print("-" * 80)
        print()

print(" Interactive chatbot ready!")
print(" Run: chat_with_aurora_bay() to start an interactive session")

 Interactive chatbot ready!
 Run: chat_with_aurora_bay() to start an interactive session


In [24]:
# Summary and statistics
summary_query = f"""
SELECT
  COUNT(*) as total_faqs,
  COUNT(DISTINCT question) as unique_questions
FROM `{project_id}.aurora_bay_faqs.aurora_bay_faqs_with_embeddings`
"""

print(" Aurora Bay RAG System Summary")
print("=" * 80)

stats = bq_client.query(summary_query).to_dataframe()
print(f" Total FAQ entries: {stats['total_faqs'].iloc[0]}")
print(f" Unique questions: {stats['unique_questions'].iloc[0]}")
print(f" Embedding model: text-embedding-005")
print(f" LLM model: gemini-2.0-flash")
print(f" Vector search: top_k=5")
print(f" Dataset location: US")
print("=" * 80)

 Aurora Bay RAG System Summary
 Total FAQ entries: 50
 Unique questions: 50
 Embedding model: text-embedding-005
 LLM model: gemini-2.0-flash
 Vector search: top_k=5
 Dataset location: US


In [25]:
# Additional test cases
print(" Running comprehensive test cases...")
print("=" * 80)

test_cases = [
    "When was Aurora Bay founded?",
    "What is the population?",
    "Tell me about the weather",
    "What services does the town provide?",
]

for i, question in enumerate(test_cases, 1):
    print(f"\n[Test {i}/{len(test_cases)}]")
    print(f"Question: {question}")
    print("-" * 40)

    answer = ask_aurora_bay(question, top_k=3)
    print(f"Answer: {answer}")
    print()

 Running comprehensive test cases...

[Test 1/4]
Question: When was Aurora Bay founded?
----------------------------------------
 Searching for relevant information...
 Found 3 relevant FAQs

 Building prompt with context...
 Generating answer with Gemini...
Answer: "Aurora Bay was founded in 1901 by a group of fur traders who recognized the region’s strategic coastal location.\n"


[Test 2/4]
Question: What is the population?
----------------------------------------
 Searching for relevant information...
 Found 3 relevant FAQs

 Building prompt with context...
 Generating answer with Gemini...
Answer: "Aurora Bay has a population of approximately 3,200 residents, although it can fluctuate seasonally due to temporary fishing and tourism workforces.\n"


[Test 3/4]
Question: Tell me about the weather
----------------------------------------
 Searching for relevant information...
 Found 3 relevant FAQs

 Building prompt with context...
 Generating answer with Gemini...
Answer: "Winters a